In [ ]:
# Setup
import os
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger

# Load configuration
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
model_id = os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16')
temperature = float(os.environ.get("TEMPERATURE", 0.0))
max_tokens = int(os.environ.get("MAX_TOKENS", 4096))

print(f"🔗 Connecting to: {base_url}")
print(f"🤖 Using model: {model_id}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")


In [ ]:
# Test direct tool call first
print("🧪 Testing direct tool call first...")
try:
    result = client.tool_runtime.invoke_tool(
        tool_name="jira_create_issue",
        kwargs={
            "project_key": "KAN",
            "summary": "Direct Test - Pre-Agent",
            "issue_type": "Task",
            "description": "Testing direct tool call before agent test",
            "additional_fields": {}
        }
    )
    print("✅ Direct tool call SUCCESS!")
    print(f"Created issue: {result}")
except Exception as e:
    print(f"❌ Direct tool call FAILED: {e}")


In [ ]:
# Create ReActAgent with FIXED configuration
print("🤖 Creating ReActAgent with FIXED configuration...")

# Sampling parameters
sampling_params = {
    "strategy": {"type": "greedy"},
    "max_tokens": max_tokens,
    "temperature": temperature,
}

# Create agent with FIXED instructions
agent = ReActAgent(
    client=client,
    model=model_id,
    instructions="""You are a Jira assistant. When asked to create a Jira issue, you must:
1. Use the jira_create_issue tool
2. Provide project_key, summary, issue_type, description, and additional_fields
3. Use exact tool names and correct parameter format
4. Always include additional_fields as a dictionary, not JSON string
5. Execute the tool call, don't just describe it""",
    tools=["mcp::atlassian"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

print("✅ ReActAgent created with FIXED configuration")


In [ ]:
# Test 1: Simple Jira Issue Creation with ReActAgent
print("🧪 Test 1: Create Simple Jira Issue with ReActAgent")
print("="*55)

try:
    # Create a session
    session_id = agent.create_session("jira-agent-test")
    print(f"✅ Session created: {session_id}")
    
    # Simple test prompt
    test_prompt = "Create a Jira issue in the KAN project with summary 'ReActAgent Test Issue' and type 'Task'"
    
    print(f"\n📝 Test prompt: {test_prompt}")
    print("\n" + "="*55)
    
    # Create turn with streaming
    response = agent.create_turn(
        messages=[{"role": "user", "content": test_prompt}],
        session_id=session_id,
        stream=True,
        toolgroups=["mcp::atlassian"]
    )
    
    print("\n🚀 Processing with ReActAgent...")
    
    # Process streaming response
    for log in EventLogger().log(response):
        log.print()
        
    print("\n✅ Test 1 completed!")
    
except Exception as e:
    print(f"❌ Test 1 failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 2: OOM Error Incident with ReActAgent
print("\n🧪 Test 2: Create OOM Error Incident with ReActAgent")
print("="*50)

try:
    # Create a new session
    session_id = agent.create_session("jira-oom-test")
    print(f"✅ Session created: {session_id}")
    
    # OOM error incident prompt
    oom_prompt = """Create a Jira incident for a pod failing due to OOM error in the KAN project:
    - Summary: 'Pod failing due to OOM error'
    - Issue Type: 'Incident'
    - Description: 'Pod experiencing Out of Memory errors causing failures'
    - Priority: High
    - Labels: ['oom-error', 'pod-failure', 'high-priority']"""
    
    print(f"\n📝 OOM Incident prompt:")
    print(oom_prompt)
    print("\n" + "="*50)
    
    # Create turn with streaming
    response = agent.create_turn(
        messages=[{"role": "user", "content": oom_prompt}],
        session_id=session_id,
        stream=True,
        toolgroups=["mcp::atlassian"]
    )
    
    print("\n🚀 Processing OOM incident creation...")
    
    # Process streaming response
    for log in EventLogger().log(response):
        log.print()
        
    print("\n✅ Test 2 completed!")
    
except Exception as e:
    print(f"❌ Test 2 failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: Verify Issues Were Created
print("\n🧪 Test 3: Verify Issues Were Created")
print("="*40)

try:
    # Search for recently created issues
    search_result = client.tool_runtime.invoke_tool(
        tool_name="jira_search",
        kwargs={"jql": "project = KAN ORDER BY created DESC"}
    )
    
    print("✅ Search completed!")
    print(f"Search result: {search_result}")
    
    # Check if our test issues were created
    if "ReActAgent Test Issue" in str(search_result) or "OOM error" in str(search_result):
        print("🎉 SUCCESS: ReActAgent created issues!")
    else:
        print("⚠️  WARNING: ReActAgent may not have created issues")
        
except Exception as e:
    print(f"❌ Search failed: {e}")


In [ ]:
# Alternative: Manual Tool Execution Test
print("\n🔧 Alternative: Manual Tool Execution Test")
print("="*45)

# If ReActAgent doesn't work, let's manually execute the tool call
print("Testing manual tool execution with ReActAgent format...")

try:
    # Create a simple issue manually using the same format ReActAgent would use
    manual_result = client.tool_runtime.invoke_tool(
        tool_name="jira_create_issue",
        kwargs={
            "project_key": "KAN",
            "summary": "Manual Test - ReActAgent Format",
            "issue_type": "Task",
            "description": "Testing manual execution with ReActAgent parameter format",
            "additional_fields": {
                "priority": {"name": "Medium"},
                "labels": ["manual-test", "react-agent-format"]
            }
        }
    )
    
    print("✅ Manual tool execution SUCCESS!")
    print(f"Created issue: {manual_result}")
    
except Exception as e:
    print(f"❌ Manual tool execution FAILED: {e}")
    import traceback
    traceback.print_exc()
